# Decisionn Trees
![](picture/01.png)

Check if every item in the dataset is in the same class:

    if so return the class label
    Else
        find the best feature to split the data
        split the dataset
        create a branch node
             for each split
                 call createBranch and add the result to the branch node
             retuen branch node
             
Please note the recursive nature of createBranch. It calls itself in the second-to-last line.

**Note:** some decision trees make a binary split of the data,but we won't do this. We'll follow the ID3 algorithm.

### 1 ID3

Entropy is defined as the expected value of the information. First, we need to define information. If you’re classifying something that can take on multiple values, the information for symbol xi is defined as

$l(x_i) = log_{2}p(x_i)$

where p(xi) is the probability of choosing this class.

To calculate entropy, you need the expected value of all the information of all pos-
sible values of our class. This is given by

$H = - \sum_{i=1}^{n}p(x_i)log_{2}p(x_i)$

where n is the number of classes.

Note: click [here](https://blog.csdn.net/acdreamers/article/details/44661149) to ID3

### DataSet

See the data in table 3.1. It contains five animals pulled from the sea and asks if they can survive without coming to the surface and if they have flippers. We would like to classify these animals into two classes: fish and not fish. Now we want to decide whether we should split the data based on the first feature or the second feature. To answer this question, we need some quantitative way of determining how to split the data. We’ll discuss that next.

![](picture/02.png)

### 1.1 Let's look how to compute SannonENT

If we target learning "is Fish?".

The table 3.1 total examples is 5, and 2 "yes",3 "no". So,we compute shannonENT as follows

$H = - \frac{3}{5} log_{2}\frac{3}{5} - \frac{2}{5} log_{2} \frac{2}{5} = 0.9709505944546686$



You’ll stop under the following conditions: you run out of attributes on which to split or all the instances in a branch are the same class. If all instances have the same class, then you’ll create a leaf node, or terminating block. Any data that reaches this leaf node is deemed to belong to the class of that leaf node. 
![](picture/03.png)

### 1.2 Befor we create the Decision Tree,we need learning decision tree's logist.

Let's look what is decision tree's logist.

We using this example dataset

![](picture/04.png)


- step 1: we need compute Entropy(Fish) = $- \frac{3}{5} log_{2}\frac{3}{5} - \frac{2}{5} log_{2} \frac{2}{5}=0.9709505944546686$
- step 2: comput IG
    - No surfacing:
        - 1:[is Fish] yes,yes,no 
        - 0:[is Fish] no,no
        - Entropy(1) = $- \frac{2}{3} log_{2}\frac{2}{3} - \frac{1}{3} log_{2} \frac{1}{3}=0.918296 $
        - Entropy(0) = $- 0 - \frac{2}{2} log_{2} \frac{2}{2}=0 $
        - Entropy(No surfacing | Fish) = $\frac{3}{5}\times 0.918296 + \frac{2}{5} \times 0 = 0.5509$
        - $\frac{3}{5},\frac{2}{5}$: in total dataset
        - IG(No surfacing | Fish) = 0.9709505944546686 - 0.5509 = 0.42
    - Flippers:
        - 1:[is Fish] yes,yes,no,no
        - 0:[is Fish] no
        - Entropy(1) = $- \frac{2}{4} log_{2}\frac{2}{4} - \frac{2}{4} log_{2} \frac{2}{4}=1.0 $
        - Entropy(0) = $- 0 - \frac{1}{1} log_{2} \frac{1}{1}=0 $
        - Entropy(Flippers | Fish) = $\frac{4}{5}\times 1 + \frac{1}{5} \times 0 = 0.8$
        - $\frac{1}{5},\frac{4}{5}$: in total dataset
        - IG(Flippers | Fish) = 0.9709505944546686 - 0.8 = 0.17095
        
- step 3: choose best IG(max IG)
    - so,we choose feature "No surfacing".
- step 4: using feature "No surfacing" to split dataset.
- step 5: repeat like setp 1-3
    
**Last Note:**
 - if best feature can't split dataset to  have same class label, then ,can using other feature to split dataset.
     - [[1, 1, 'yes'], [1, 0, 'yes'], [1, 0, 'no'], [0, 1, 'yes'], [0, 1, 'no']]
 
 - if case have no more features to split, but we also can't split dataset to have same class label, then we use "majority vote."
     - [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'yes'], [0, 1, 'no']]

### 2 Build Decision Trees

在构建之间我们需要明白几件事:

- 1.在代码中是需要计算标签的经验熵,直接选取各个特征下的信息熵最小的也就是信息增益最大的特征,因为我们做的是$IG=H(y)-H(features)$

- 2.每次计算各个feature的不同熵的时候,实际上还是计算各个feature划分后数据的labels信息熵

- 3.为了计算方便,我们会使用[pandas](https://pandas.pydata.org/)

In [1]:
import numpy as np
import pandas as pd

#### 2.2 load data set

In [2]:
def loadDataset():
    """
    Load dataset
    
    Returns:
    -------
        dataSet: data set
        labels: data label
    Note:
    ----
        y: yes
        n: no
        last columns is labels.
    """
    dataSet = [['y','y','y'],
              ['y','y',"y"],
              ['y','n',"n"],
              ['n','y',"n"],
              ['n','y',"n"]]
    labels = ['no surfacing','flippers','lables']
    
    return dataSet,labels

In [3]:
dataSet,labels = loadDataset()

现在我们将其转换成pandas的形式:

```python
pd.DataFrame```

In [4]:
data = pd.DataFrame(dataSet,columns=labels)
data

,no surfacing,flippers,lables
0,y,y,y
1,y,y,y
2,y,n,n
3,n,y,n
4,n,y,n


#### 2.3  Calculate Shannon Ent at labels.

虽然我们现在不需要计算整体样本的信息熵(H(labels)),但是我们知道,计算各个特征的信息熵实际上是用各个特征的子特征拆分数据集,统计数量的还是labels列,所以我们需要定义计算labels信息熵的函数

In [5]:
def calcShannonEnt_labels(data):
    """
    Calculate ShannonEnt at labels.
    Parameters:
    ----------
        data: pandas of data
    Return:
    ------
        Ent_H_y: ShannonEnt of label.
    """
    allData_size = data.index.size
    labels = data.iloc[:,-1]
    labels_dict = {}
    H_y = []
    for label in labels:
        if label not in labels_dict:
            labels_dict[label] = 1
        else:
            labels_dict[label] += 1
        
    for key,value in labels_dict.items():
        H_y.append(value)
    
    H_y_arr = np.array(H_y) /allData_size
    Ent_H_y = np.sum(-H_y_arr * np.log2(H_y_arr))
    
    return Ent_H_y

In [6]:
Ent_H_y = calcShannonEnt_labels(data)
print('Ent of data set labels:',Ent_H_y)

Ent of data set labels: 0.9709505944546686


#### 2.4 Calculate Shannon Ent at features.

现在开始进行计算所有的特征的香农熵:

(0) 先来回顾不同特征的香农熵是如何计算的:

No surfacing:
- 1:[is Fish] yes,yes,no 
- 0:[is Fish] no,no
- Entropy(1) = $- \frac{2}{3} log_{2}\frac{2}{3} - \frac{1}{3} log_{2} \frac{1}{3}=0.918296 $
- Entropy(0) = $- 0 - \frac{2}{2} log_{2} \frac{2}{2}=0 $
- Entropy(No surfacing | Fish) = $\frac{3}{5}\times 0.918296 + \frac{2}{5} \times 0 = 0.5509$
- $\frac{3}{5},\frac{2}{5}$: in total dataset

(1) 使用不同特征划分样本集,由于我们数据定义的最后一列是labels,所以:

```python
data.columns[:-1]```

(2) 不同特征下的不同子特征划分需要使用到:

```python
data[data[feature]==feature_son]```

(3) 可以使用我们之间定义的calcShannonEnt_labels来计算使用各个特征拆分完成的数据集去计算labels的香农熵.

```python
calcShannonEnt_labels(feature_data_son)```

(4) 计算单个特征的信息熵需要使用总体样本个数和使用单个特征拆分后各个子特征的个数作为权重:

定义:```all_data_size = data.index.size```为整体样本个数(也就是$\frac{3}{5},\frac{2}{5}$的分母部分)

定义:```fenzi_size = feature_data_son.index.size```为各个特征下的子特征的个数(也就是$\frac{3}{5},\frac{2}{5}$的分子部分)

(5) 计算各个特征下子特征的信息熵Ent_sum,放入best_feature_dict保存,也就是说best_feature_dict中存放的就是各个特征的信息熵.

(6) 最后以升序排列字典,那么最小的特征熵,也就是信息增益最大的特征.
    

In [7]:
def calcShannonEnt_feature(data):
    """
    Implementation calculate ShannonEnt at feature.
    
    Prameters:
    ---------
        data: pandas of data.
    Returns:
    -------
        best_feature: IGmax feature.
    """
    
    all_data_size = data.index.size # get all data size. like 5 in this example.
    features_name = data.columns[:-1] # get the feature name. like 2 : 'no surfacing','flippers'
    
    best_feature_dict = {} # cache shannon ent with every feature.
    
    for feature in features_name: # star loop feature name.
        unique_feature_son = pd.unique(data[feature]) # get subfeatures
        
        Ent_sum = 0
        
        for feature_son in unique_feature_son: # loop subfeatures.
            
            feature_data_son = data[data[feature]==feature_son]# like data[data['no surfacing']=='y']
            fenzi_size = feature_data_son.index.size # get molecular part
            
            # using function calcShannonEnt_labels to calculate shannon Ent
            Ent_feature_son = calcShannonEnt_labels(feature_data_son)
            
            # weights * Ent feature_son:like 3/5 * 0.918296
            Ent_sum += Ent_feature_son * fenzi_size / all_data_size

        best_feature_dict[feature] = Ent_sum # get single feature shannon ENT.
    
    best_feature = sorted(best_feature_dict.items(),key=lambda z:z[1]) # sort and get minium ent.
    
    return best_feature[0][0]
    

In [8]:
best_feature = calcShannonEnt_feature(data)
best_feature

'no surfacing'

可以看到,第一次选择的最优划分特征为:no surfacing

得到了最优特征之后我们需要拆分数据集

#### 2.5 Split Trees

以最优的划分特征来划分数据集,这里需要注意的有一下几点:

(1)我们需要统计每次划分使用了多少数据```sum_f```,作为下一部分的退出条件使用(稍后会说明).

(2)以最优的特征条件下的子特征划分会有两种结果:

(2.1) ```label_size == 1```: 也就是说划分后的节点为叶结点

(2.2) 划分后的节点为非叶结点

其中非叶结点需要再次调用calcShannonEnt_feature函数对于剩余部分计算最优特征.

所以我们定义:

当为叶结点的时候,我们无需再次计算calcShannonEnt_feature,而对于非叶结点,我们给予非叶结点的数据以便于再次选取最优特征.

这里需要注意的是,对于非叶结点再次计算最优特征,**数据中不能涵盖已经使用过的最优特征**.即:

```python
data_son.drop(columns=best_feature)```

最后次函数将返回两种值:

(1) 列表长度为2表示为叶结点,无需再次选取最优节点

(2) 列表长度为3表示非叶结点,需要在剩余的数据中再次选择最优的特征.




In [9]:
def split_Trees(data,best_feature):
    """
    split data.
    Parameters:
    ----------
        data: pandas data
        best_feature: best feature
    
    """
    unique_bf = pd.unique(data[best_feature])
    Split_result = []
    sum_f = 0
    for feature_son in unique_bf:
        data_son = data[data[best_feature]==feature_son] # using subfeature to split data.
        label_size = pd.unique(data_son.iloc[:,-1]).size
        
        if label_size == 1:
            sum_f += data_son.index.size
            Split_result.append([best_feature,feature_son])
        else:
            drop_data_son = data_son.drop(columns=best_feature) # drop used best feature.
            Split_result.append([best_feature,feature_son,drop_data_son])
    
    return Split_result,sum_f

In [10]:
Split_result,sum_f = split_Trees(data,best_feature)
print('split result is:',Split_result)
print('-----------------------------')
print('using number of data in best feature:',sum_f)

split result is: [['no surfacing', 'y',   flippers lables
0        y      y
1        y      y
2        n      n], ['no surfacing', 'n']]
-----------------------------
using number of data in best feature: 2


很明显可以看出最优特征'no surfacing'下的子特征'y'需要再次选取最优条件且再次计算的数据为后面部分(没有no surfacing,因为该特征已经使用过了.)

### 2.6 Build DT(ID3)

现在开始搭建DT.

由于决策树是一种递归,或者说是重复挑选最优特征的过程,所以我们需要定义迭代退出条件:

(1) 所有原始样本个数已经被划分完毕

这里的**已经被划分完毕**,需要在叶结点下统计,因为只有叶结点才会消耗原始来本数量,非叶结点需要再次挑选最优特征,所以不消耗原始样本特征.

也就是split_Trees函数的sum_f.

(2) 所有特征已经使用完毕:

(2.1) 所有特征使用完毕且样本也已经完全划分完毕

(2.2) 所有特征使用完毕,但是样本依然没有划分完毕,这种情况我们一般是对于剩余不能划分完毕的样本取类别出现最多的当做预测类别即函数prob_label


In [11]:
def prob_label(data,REST):
    """
    When 2.2
    
    Parameters:
    -----------
        data:pandas data
        REST: Contains all leaf nodes
    Return:
    ------
        feature_prob: maximum prob label.
    """
    sumOflabels = {}
    
    # 获取最后未能完全划分的样本
    for feature,feature_son in REST:
        data = data[data[feature] != feature_son]
    
    data_labels = data.iloc[:,-1]
    # 统计未能完全划分数据的标签个数
    for label in data_labels:
        if label not in sumOflabels:
            sumOflabels[label] = 1
        else:
            sumOflabels[label] += 1
            
    # 取出现次数最多的标签作为概率标签
    prob_label = sorted(sumOflabels.items(),key=lambda x:x[1],reverse=True)[0][0]
    feature_prob = [feature,'!'+feature_son,prob_label] 
     
    return feature_prob

In [12]:
def DT(data):
    """
    build DT
    Parameters:
    ----------
        data: pandas data
    Return:
    ------
        REST: Contains all leaf nodes
        bool: 
            True:样本划分完毕
            False: 样本未划分完毕
    """
    REST = [] # 存放所有的叶结点
    ALL_data_size = data.index.size # 获得样本总数
    ALL_data_features = data.columns[:-1].size # 获取所有特征数
    sum_data = 0
    sum_feature = 0
    while 1:
        # 1.所有样本都已经使用完毕
        if sum_data == ALL_data_size:
            print('所以样本划分完毕')
            return REST,True
        # 2.所有特征使用完毕；2.1 样本使用完毕,2.2样本未划分完毕
        if sum_feature == ALL_data_features:
            if sum_data == ALL_data_size:
                print('样本划分完毕')
                return REST,True
            else:
                print('特征已经使用完毕,但是样本依然没有完全划分')
                return REST,False
        best_feature = calcShannonEnt_feature(data)
        sum_feature +=1 # 每得到一个特征 使用特征自加1
        Split_result,sum_f = split_Trees(data,best_feature)
        sum_data += sum_f
        for i in Split_result:
            if len(i) == 2: # 判断为叶结点的情况
                REST.append(i)
            else: # 非叶结点的情况
                data = i[-1] # 更新需要再次选择最优条件的数据.

In [13]:
def DecisionTrees(data):
    
    REST,is_ok = DT(data)
    if not is_ok:
        feature_prob = prob_label(data,REST)
        REST.append(feature_prob)
        return REST
    else:
        return REST

In [14]:
DecisionTrees(data)

所以样本划分完毕


[['no surfacing', 'n'], ['flippers', 'y'], ['flippers', 'n']]

我们更改一下数据集就可以获得特征使用完毕,但是样本为划分完毕的情况:

In [15]:
dataSet = [['y','y','y'],
              ['y','y',"n"] , # 更改此处的labels从y变为n
              ['y','n',"n"],
              ['n','y',"n"],
              ['n','y',"n"]]
labels = ['no surfacing','flippers','lables']

In [16]:
data_ = pd.DataFrame(dataSet,columns=labels)
DecisionTrees(data_)

特征已经使用完毕,但是样本依然没有完全划分


[['no surfacing', 'n'], ['flippers', 'n'], ['flippers', '!n', 'y']]

这里的'!n'表示'flippers'特征下的子特征如果不是'n',那么结果大概率是'y'.

### 3.C4.5

**信息增益比:**

其信息增益与训练数据集的经验熵$H(D)$之比.

由于C4.5在生成过程中是采用信息增益比来计算选择特征,所以这里就不在多说了.

---------------

### 4.CART

CART决策树的生成即使递归地构建二叉决策树的过程,对回归树用平方误差最小化准则,对分类树用基尼指数(Gini index)最小化准则,进行特征选取，生成二叉树.

CART生成算法

输入:训练数据集D,停止计算的条件

输出:CART决策树

根据训练数据集,从根节点开始,递归地对每个节点进行一下操作,构建二叉树决策树:

(1) 设节点的训练数据集为D,计算现有特征对该数据集的Gini指数.此时,对每个特征A,对其可能取的每个值a,根据样本点A=a的测试为"是","否"将D分割成D1和D2两个部分,使用Gini计算A=a的Gini指数

(2) 在所有可能的特征A以及它们所有可能的切分点a中,选择Gini指数最小的特征以及对应切分点作为最优特征和最优切分点.依最优特征与最优切分点,从现节点生成两个子节点,将训练数据集依特征分配到两个子节点中去.

(3) 对两个子节点递归地调用(1),(2)直到满足停止条件

(4) 生成CART


**注意:**

- $Gini(D,A)=\frac{|D_1|}{|D|}Gini(D_1)+\frac{|D_2|}{|D|}Gini(D_2)$
- $Gini(D) = 1-\sum_{k=1}^{K}(\frac{|C_k|}{|D|})^2$
- $C_k$是$D$属于第k类的样本子集,K是类的个数

- 停止条件:
    - 结点中的样本个数小于阈值.
    - 样本集的Gini指数小于预订阈值,也就是说样本基本属于同一类.
    - 该次样本集中没有更多的特征
    

In [17]:
import pandas as pd
import numpy as np

### 4.1 逻辑阐述

1.为了实现CART,这里主要使用pandas这个库，方便处理训练样本

2.刚开始调用训练的时候需要初始化一些参数

```python
self.m,self.n = data.shape 
self.Dtree = {}
self._Gini = np.Inf
```

3.在参数初始化完毕之后,需要先定义递归退出条件

```python

if  m <= self.Threshold_data:
    print("第一个结束条件退出")
return self.Dtree
...

```

4.接下去就是计算Gin指数获取最优的特征A以及对应的切分点a.

```python
def calcGini(self,X,labels):
    ....
```

**注意:**这里的计算直接采用向量的形式(因为计算Gini的公式可以转换为向量的形式),并且将$Gini(D,A)$拆分成两个部分

5.在计算出最优的Gini得出最优的特征A以及对应的切分点a之后,我们需要依照A=a进行切分

```python
def split_Tree(self,X,best_label,best_class):
    ...
```

**注意:**虽然我们可以使用A=a进行划分数据集,但是我们并不知道是切分点a的那一侧是叶节点,比如说可能是a的一侧是叶结点,也可能是除了a之外的分割是叶结点,还可能由a划分之后的两个子节点都不是叶结点,还可能是由a分割之后两侧都是叶结点所以我们需要做判断

```python
if ((X_equal_last_label == X_equal_last_label.iloc[0]).all())...:
    ....
```


6.在数据集划分完毕之后,我们依照上面4中情况进行处理:

- 1.A=a划分后两侧都是叶结点,那么可以将其放入最终的字典内
- 2.A=a划分后,某一侧是叶结点,但是另外一侧不是,那么我么将是叶结点的那一侧放入最终字典,将不是叶结点的那一侧继续递归
- 3.A=a划分后,两侧都不是叶结点的话,那么继续递归,为了结果好看,会创建一个列表来放入子两侧节点的划分结果

```python

if (res_1 is False) and (res_2 is False)...:
    ...

```

7.在整体递归结束后,就可以得到最终的字典,另外我们也可以设置停止条件中的阈值来处理过拟合的情况,当然剪枝是最好的操作


---------------------

In [18]:
class CART:
    def __init__(self,Threshold_data=0,Threshold_Gini=0.001,isPrintGini=False):
        
        self.isPrintGini = isPrintGini
        self.Threshold_data = Threshold_data
        self.Threshold_Gini = Threshold_Gini
        self.is_start = True
        self.cache_best_label = None
        
    def init_args(self,data,labels):
        self.m,self.n = data.shape
        self.Dtree = {}
        self._Gini = np.Inf
        
    def split_Tree(self,X,best_label,best_class):
        """
        检查叶节点,由于Gini计算出来的best_label,我们不知道到底那一侧是叶节点,所以需要分开来判定
        """
        X_equal_ = X[X[best_label]==best_class]
        X_not_equql_ = X[X[best_label]!=best_class]
        X_equal_last_label = X_equal_.iloc[:,-1]
        X_not_equql_last_label = X_not_equql_.iloc[:,-1]
        
        if ((X_equal_last_label == X_equal_last_label.iloc[0]).all()) and ((X_not_equql_last_label == X_not_equql_last_label.iloc[0]).all()):
            # 说明X_equal_和X_not_equql_为叶节点,也就是说两侧都是叶节点,那么该分支划分完毕
            
            return False,False
        
        elif (X_equal_last_label == X_equal_last_label.iloc[0]).all():
            #说明X_equal_l是叶节点,那么另一侧就不是应该继续划分
            return X_not_equql_,False
        
        
        elif (X_not_equql_last_label == X_not_equql_last_label.iloc[0]).all():
            #说明X_not_equql_是叶节点,那么另一侧就不是应该继续划分  
            return False,X_equal_
        
        else:
            # 说明两侧都不是,那么两侧都应该继续划分
            return X_equal_,X_not_equql_
        
                
    def calcGini(self,X,labels):
        res_list = []
        for label in labels[:-1]:
                
            dupli_X = X.drop_duplicates(label)[label]
            groupby_X = X.groupby([label,labels[-1]]).size()
            
            for i in dupli_X:
     
                #######Gini 第一项 D_1#####
                gini_1 = 1-np.sum(np.power(groupby_X[i] /X.groupby([label]).size()[i],2))
                Gini_1 = X.groupby([label]).size()[i] / self.m * gini_1
                
                ########## Gini 第二项 D_2 ########
                filter_X = X[X[label] != i]  # 过滤掉含有i的元素
                gini_2 = 1- np.sum(np.power(filter_X.groupby(labels[-1]).size() / filter_X.groupby(labels[-1]).size().sum(),2))
                Gini_2 = (self.m - X.groupby([label]).size()[i]) / self.m  * gini_2
                GINI = Gini_1 + Gini_2
                
                # 放入列表,用于比较最小Gini值
                res_list.append([label,i,GINI])
                    
                    
        # 比较最小Gini值
        best_label,best_class,_Gini = sorted(res_list,key=lambda x:x[2])[0]
        
        return best_label,best_class,_Gini
        
    
    def input_Dtree(self,best_label,best_class):
        
        if self.cache_best_label is not None:
            self.Dtree[self.cache_best_label].append({best_label:best_class})
        else:
            self.Dtree[best_label] = best_class
    
    
    def fit(self,data,labels):
        m = data.shape[0]
        
        if self.is_start:
            self.init_args(data,labels)
            self.is_start = False
        
        # 定义停止条件
        if  m <= self.Threshold_data:
            print("第一个结束条件退出")
            return self.Dtree
        
        elif self._Gini <= self.Threshold_Gini:
            print("第二个结束条件退出")
            
            return self.Dtree
        elif len(labels)-1 == 0:
            print("第三个结束条件退出")
            
            return self.Dtree
        
        else:
            best_label,best_class,self._Gini = self.calcGini(data,labels)
                
            if self.isPrintGini:
                print("~Best label:",best_label,"~Best class:",best_class,"~Caculate Gini:",self._Gini)
                
            # 开始划分,如果有一个是叶节点,那么直接是{key:value}的形式,否则{key:{key:{}...}}的形式
            res_1,res_2 = self.split_Tree(data,best_label,best_class)
            
            # 依照A=a不同情况进行结果处理
            if (res_1 is False) and (res_2 is False):
                
                self.input_Dtree(best_label,best_class)
                
            elif (res_1 is not False) and (res_2 is False):
                
                self.input_Dtree(best_label,best_class)
                    
                data = res_1.drop(columns=[best_label])
                labels = data.columns
                self.fit(data,labels)
                
            elif (res_1 is False) and (res_2 is not False):
                
                self.input_Dtree(best_label,best_class)
                    
                data = res_2.drop(columns=[best_label])
                labels = data.columns
                self.fit(data,labels)
                
            elif (res_1 is not False) and (res_2 is not False):
                
                self.Dtree[best_label+":"+best_class] = []
                self.cache_best_label = best_label+":"+best_class
                print("{}:{} 子分支第一个".format(best_label,best_class))
                
                data = res_1
                labels = data.columns
                self.fit(data,labels)
                
                self.Dtree[best_label+":not "+best_class] = []
                self.cache_best_label = best_label+":not "+best_class
                
                print("{}:not {} 子分支第二个".format(best_label,best_class))
                
                data = res_2
                labels = data.columns
                self.fit(data,labels)
                self.cache_best_label = None
                
        return self.Dtree


### 4.2 尝试统计学习方法(李航)例子5.4

In [19]:
# 书上题目5.1
def create_data():
    datasets = np.array([
               ['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ])
    labels = np.array(['年龄', '有工作', '有自己的房子', '信贷情况', '类别'])
    # 返回数据集和每个维度的名称
    return datasets, labels

In [20]:
datasets, labels = create_data()

In [21]:
data = pd.DataFrame(data=datasets,columns=labels)
data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [22]:
cart2 = CART(isPrintGini=True)    

In [23]:
data_test = pd.DataFrame(data=datasets,columns=labels)
Dtree = cart2.fit(data_test,labels)
print(Dtree)

~Best label: 有自己的房子 ~Best class: 否 ~Caculate Gini: 0.26666666666666666
~Best label: 有工作 ~Best class: 否 ~Caculate Gini: 0.0
{'有自己的房子': '否', '有工作': '否'}


可以看到结果是正确的

### 4.3 再来尝试机器学习实战中的案例

In [24]:
def loadingDataSet():
    """
    Implement predict contact lens
    returns:
        lenesTree
    """
    path = "data_set/lenses.txt"
    fr = open(path)
    lenses = np.array([inst.strip().split('\t') for inst in fr.readlines() ])
    lenesLabels = ['age','prescript','astigmatic','tearRate','Joker']
    return lenses,lenesLabels

In [25]:
lenses,lenesLabels = loadingDataSet()
data_lenses = pd.DataFrame(data=lenses,columns=lenesLabels)
data_lenses

,age,prescript,astigmatic,tearRate,Joker
0,young,myope,no,reduced,no lenses
1,young,myope,no,normal,soft
2,young,myope,yes,reduced,no lenses
3,young,myope,yes,normal,hard
4,young,hyper,no,reduced,no lenses
5,young,hyper,no,normal,soft
6,young,hyper,yes,reduced,no lenses
7,young,hyper,yes,normal,hard
8,pre,myope,no,reduced,no lenses
9,pre,myope,no,normal,soft


In [26]:
cart2 = CART(isPrintGini=True,Threshold_Gini=-np.Inf) 

In [27]:
Dtree = cart2.fit(data_lenses,lenesLabels)
print(Dtree)

~Best label: tearRate ~Best class: reduced ~Caculate Gini: 0.3263888888888889
~Best label: astigmatic ~Best class: yes ~Caculate Gini: 0.31944444444444436
astigmatic:yes 子分支第一个
~Best label: prescript ~Best class: hyper ~Caculate Gini: 0.05555555555555555
~Best label: age ~Best class: young ~Caculate Gini: 0.0
astigmatic:not yes 子分支第二个
~Best label: age ~Best class: presbyopic ~Caculate Gini: 0.041666666666666664
~Best label: prescript ~Best class: myope ~Caculate Gini: 0.0
{'tearRate': 'reduced', 'astigmatic:yes': [{'prescript': 'hyper'}, {'age': 'young'}], 'astigmatic:not yes': [{'age': 'presbyopic'}, {'prescript': 'myope'}]}


原题分割结果如下:
![](picture/tree_1.png)

可以看出结果是一模一样的,但是这里不够的是,并没有采取剪枝措施.

CART剪枝:**统计学方法**P73

### 5. sklearn.tree.DecisionTreeClassifier

[sklearn.tree.DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

[sklearn.tree.DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor)

In [29]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
import graphviz

In [30]:
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    # print(data)
    return data[:,:2], data[:,-1]
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

注意,由于scikit使用的数据形式与之前我们创建的形式有差异,所以我们并不能使用scikit去检验我们例子的结果,当然你也可以更改例子中的数据形式来尝试

scikit 数据形式:

```
X : array-like or sparse matrix, shape = [n_samples, n_features]
The training input samples. Internally, it will be converted to dtype=np.float32 and if a sparse matrix is provided to a sparse csc_matrix.

y : array-like, shape = [n_samples] or [n_samples, n_outputs]
The target values (class labels) as integers or strings.
```



In [31]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [32]:
clf.score(X_test, y_test)

0.9666666666666667

In [33]:
export_graphviz(clf, out_file="picture/mytree.dot")

将dot转换为png文件需要安装[graphviz](https://www.graphviz.org/download/)
使用[sklearn.tree.export_graphviz](https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html)中的

```
$ dot -Tps tree.dot -o tree.ps  (PostScript format) 
$ dot -Tpng tree.dot -o tree.png    (PNG format)```

![](picture/mytree.png)